# Modelamiento
En este notebook se realizará el entrenamiento y validación de los modelos de riesgo crediticio.

## Cargue de librerías y parámetros

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agrega la ruta del directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar los módulos
from procesamiento_datos import *
from modelado import *
from evaluacion import *

# Importar bibliotecas adicionales para modelamiento
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

## Cargue de datos preprocesados

In [ ]:
# Cargar datos preprocesados
df = pd.read_csv('../data/processed/datos_modelado.csv')

# Convertir la fecha a datetime
df['fecha_cierre'] = pd.to_datetime(df['fecha_cierre'])

print("Dimensiones del dataset:", df.shape)
print("\nColumnas disponibles:")
print(df.columns.tolist())
print("\nDistribución de la variable objetivo:")
print(df['default'].value_counts(normalize=True) * 100)

## Preparación de datos para modelamiento

In [ ]:
# Seleccionar features para el modelo
features_numericas = ['dias_mora', 'score_credito', 'promedio_dias_mora', 
                     'porcentaje_pagos_tiempo', 'monto_promedio_facturas',
                     'ratio_pago_promedio', 'volatilidad_dias_mora',
                     'facturas_vencidas_actuales', 'saldo_pendiente_total',
                     'ipc', 'pib', 'tasa_credito', 'salario_minimo']

features_categoricas = ['categoria_riesgo', 'calificacion', 'nombre_linea']

# Preparar features numéricas
X_num = df[features_numericas].copy()

# Normalizar variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)

# Preparar features categóricas (one-hot encoding)
X_cat = pd.get_dummies(df[features_categoricas])

# Combinar features numéricas y categóricas
X = pd.concat([X_num_scaled, X_cat], axis=1)
y = df['default']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensiones de los conjuntos de datos:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# Verificar proporción de clases en los conjuntos
print("\nDistribución de clases:")
print("Train:", pd.Series(y_train).value_counts(normalize=True))
print("Test:", pd.Series(y_test).value_counts(normalize=True))

## Entrenamiento de modelos

In [ ]:
# Entrenar modelo de Regresión Logística
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

# Entrenar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Entrenar XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Guardar los modelos entrenados
modelos = {
    'logistic_regression': lr_model,
    'random_forest': rf_model,
    'xgboost': xgb_model
}

# Crear directorio para modelos si no existe
os.makedirs('../models', exist_ok=True)

# Guardar modelos
for nombre, modelo in modelos.items():
    joblib.dump(modelo, f'../models/{nombre}_model.pkl')
    print(f"Modelo {nombre} guardado en ../models/{nombre}_model.pkl")

## Evaluación de modelos

In [ ]:
# Evaluar los modelos
for nombre, modelo in modelos.items():
    print(f"\nEvaluación del modelo {nombre}:")
    metricas = evaluar_modelo(modelo, X_test, y_test, nombre)

## Análisis de importancia de variables

In [ ]:
# Analizar importancia de variables
analizar_importancia_variables({
    'random_forest': rf_model,
    'xgboost': xgb_model
}, X_test, feature_names=X.columns)

# Modelamiento de Riesgo de Default

Este notebook se enfoca en el desarrollo y entrenamiento de modelos predictivos para identificar el riesgo de default en clientes.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, classification_report
import matplotlib.pyplot as plt
import seaborn as sns



## Carga y Preparación de Datos

Cargaremos el dataset procesado y realizaremos la preparación necesaria para el modelamiento.

In [7]:
# Cargar datos procesados
df = pd.read_csv('../data/processed/datos_modelado.csv')

# Seleccionar features para el modelo
features_numericas = [
    'score_credito', 'promedio_dias_mora', 'porcentaje_pagos_tiempo',
    'monto_promedio_facturas', 'ratio_pago_promedio', 'volatilidad_dias_mora',
    'maximo_dias_mora', 'facturas_vencidas_actuales', 'saldo_pendiente_total',
    'ipc', 'pib', 'tasa_credito', 'salario_minimo',
    'var_ipc', 'var_pib', 'var_tasa_credito', 'var_salario_minimo'
]

features_categoricas = ['categoria_riesgo', 'id_negocio']

# Preparar X e y
X = df[features_numericas + features_categoricas].copy()
y = df['default']

# Codificar variables categóricas
X = pd.get_dummies(X, columns=features_categoricas)

print("Dimensiones del dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")

# Verificar balance de clases
print("\nDistribución de la variable objetivo:")
print(y.value_counts(normalize=True))

Dimensiones del dataset:
X: (15916, 844)
y: (15916,)

Distribución de la variable objetivo:
default
0    0.876791
1    0.123209
Name: proportion, dtype: float64


## Preprocesamiento y División de Datos

Escalaremos las variables numéricas y dividiremos los datos en conjuntos de entrenamiento y prueba.

In [8]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar variables numéricas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Obtener columnas numéricas (excluyendo las dummies)
columnas_numericas = X_train.select_dtypes(include=['float64', 'int64']).columns

# Aplicar escalado
X_train_scaled[columnas_numericas] = scaler.fit_transform(X_train[columnas_numericas])
X_test_scaled[columnas_numericas] = scaler.transform(X_test[columnas_numericas])

# Guardar los datos procesados
X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("Dimensiones de los conjuntos de datos:")
print(f"X_train: {X_train_scaled.shape}")
print(f"X_test: {X_test_scaled.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

Dimensiones de los conjuntos de datos:
X_train: (12732, 844)
X_test: (3184, 844)
y_train: (12732,)
y_test: (3184,)


## Limpieza de Datos

Antes de escalar los datos, nos aseguraremos de que no haya valores faltantes o infinitos.

In [ ]:
# Función para verificar datos
def verificar_datos(X, nombre="Dataset"):
    nas = X.isna().sum()
    nas_total = nas.sum()
    inf_count = np.isinf(X.select_dtypes(include=np.number)).sum().sum()
    
    print(f"\nVerificación de {nombre}:")
    if nas_total > 0:
        print("\nColumnas con valores faltantes:")
        print(nas[nas > 0])
    print(f"\nTotal de valores faltantes: {nas_total}")
    print(f"Valores infinitos: {inf_count}")

# Función para limpiar datos
def limpiar_datos(X):
    X_clean = X.copy()
    # Reemplazar infinitos con NaN
    X_clean = X_clean.replace([np.inf, -np.inf], np.nan)
    # Para cada columna, imputar con la mediana si hay valores faltantes
    for col in X_clean.columns:
        if X_clean[col].isna().any():
            mediana = X_clean[col].median()
            X_clean[col] = X_clean[col].fillna(mediana)
    return X_clean

# Verificar estado inicial
print("Estado inicial de los datos:")
verificar_datos(X_train, "X_train")
verificar_datos(X_test, "X_test")

# Limpiar datos
X_train = limpiar_datos(X_train)
X_test = limpiar_datos(X_test)

# Verificar después de la limpieza
print("\nDespués de la limpieza:")
verificar_datos(X_train, "X_train")
verificar_datos(X_test, "X_test")

## Escalado de Datos

Ahora que los datos están limpios, procedemos con el escalado.

In [ ]:
# Escalar variables numéricas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Obtener columnas numéricas (excluyendo las dummies)
columnas_numericas = X_train.select_dtypes(include=['float64', 'int64']).columns

# Aplicar escalado
X_train_scaled[columnas_numericas] = scaler.fit_transform(X_train[columnas_numericas])
X_test_scaled[columnas_numericas] = scaler.transform(X_test[columnas_numericas])

# Verificar que no haya valores faltantes o infinitos después del escalado
print("Verificación final después del escalado:")
verificar_datos(X_train_scaled, "X_train_scaled")
verificar_datos(X_test_scaled, "X_test_scaled")

# Guardar los datos procesados y limpios
X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("\nDimensiones finales de los conjuntos de datos:")
print(f"X_train: {X_train_scaled.shape}")
print(f"X_test: {X_test_scaled.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

## Limpieza de Datos

Verificaremos y limpiaremos valores faltantes o infinitos en nuestros datos antes del entrenamiento.

In [10]:
# Verificar valores faltantes o infinitos
def verificar_datos(X, nombre="Dataset"):
    print(f"\nVerificación de {nombre}:")
    print(f"Valores faltantes: {X.isna().sum().sum()}")
    print(f"Valores infinitos: {np.isinf(X.select_dtypes(include=np.number)).sum().sum()}")
    
verificar_datos(X_train_scaled, "X_train_scaled")
verificar_datos(X_test_scaled, "X_test_scaled")

# Reemplazar valores infinitos con NaN
X_train_scaled = X_train_scaled.replace([np.inf, -np.inf], np.nan)
X_test_scaled = X_test_scaled.replace([np.inf, -np.inf], np.nan)

# Imputar valores faltantes con la mediana
for col in X_train_scaled.columns:
    if X_train_scaled[col].isna().any():
        mediana = X_train_scaled[col].median()
        X_train_scaled[col] = X_train_scaled[col].fillna(mediana)
        X_test_scaled[col] = X_test_scaled[col].fillna(mediana)

# Verificar después de la limpieza
verificar_datos(X_train_scaled, "X_train_scaled (después de limpieza)")
verificar_datos(X_test_scaled, "X_test_scaled (después de limpieza)")


Verificación de X_train_scaled:
Valores faltantes: 10788
Valores infinitos: 0

Verificación de X_test_scaled:
Valores faltantes: 2583
Valores infinitos: 0

Verificación de X_train_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_test_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_train_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_test_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0


## Entrenamiento de Modelos (con datos limpios)

Ahora entrenaremos los modelos con los datos limpios.

In [11]:
# Importar joblib para guardar los modelos
import joblib

# Crear y entrenar los modelos
models = {
    'logistic': LogisticRegression(class_weight='balanced', max_iter=1000),
    'random_forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'xgboost': xgb.XGBClassifier(scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),
                                random_state=42)
}

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    # Entrenamiento
    print(f"\nEntrenando {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    
    # Predicciones
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Guardar resultados
    results[name] = {
        'cv_scores': cv_scores,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'test_roc_auc': roc_auc
    }
    
    print(f"{name}:")
    print(f"CV ROC-AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std()*2:.3f})")
    print(f"Test ROC-AUC: {roc_auc:.3f}")

# Crear directorio para modelos si no existe
import os
os.makedirs('../models', exist_ok=True)

# Guardar los modelos
for name, model in models.items():
    joblib.dump(model, f'../models/{name}_model.pkl')

print("\nModelos guardados en el directorio '../models'")


Entrenando logistic...
logistic:
CV ROC-AUC: 0.941 (+/- 0.007)
Test ROC-AUC: 0.946

Entrenando random_forest...
logistic:
CV ROC-AUC: 0.941 (+/- 0.007)
Test ROC-AUC: 0.946

Entrenando random_forest...
random_forest:
CV ROC-AUC: 0.980 (+/- 0.006)
Test ROC-AUC: 0.987

Entrenando xgboost...
random_forest:
CV ROC-AUC: 0.980 (+/- 0.006)
Test ROC-AUC: 0.987

Entrenando xgboost...
xgboost:
CV ROC-AUC: 0.972 (+/- 0.003)
Test ROC-AUC: 0.975
xgboost:
CV ROC-AUC: 0.972 (+/- 0.003)
Test ROC-AUC: 0.975

Modelos guardados en el directorio '../models'

Modelos guardados en el directorio '../models'
